Para o uso do código, é necessário possuir uma conta na plataforma Copernicus. Caso não possua, crie a mesma aqui: https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/auth?client_id=cdse-public&response_type=code&scope=openid&redirect_uri=https%3A//dataspace.copernicus.eu/account/confirmed/1

##Download de pacotes

In [3]:
# 1. Instala pacotes necessários
!pip install openeo geopandas shapely fiona --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.6/56.6 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.7/309.7 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 91.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 45.2 MB/s eta 0:00:00


##Configurações

In [4]:
import openeo
import geopandas as gpd
from shapely.geometry import mapping
from datetime import datetime, timedelta
import os
from google.colab import drive
import shutil

In [5]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
# 1. Conectando à plataforma Copernicus Data Space com openEO - Clique no link e insira seu email e senha cadastrados na plataforma Copernicus
conn = openeo.connect("openeo.dataspace.copernicus.eu").authenticate_oidc()

Visit https://identity.dataspace.copernicus.eu/auth/realms/CDSE/device?user_code=RSBT-HWMB 📋 to authenticate.

✅ Authorized successfully

Authenticated using device code flow.


In [7]:
# 2. Lê o shapefile e converte para GeoJSON (deve estar em EPSG:4326)
shapefile_path = "/content/drive/MyDrive/Dissertacao/areas/fazenda_OuroFino2/AOI_ourofino.shp" # Altere o arquivo com base no caminho do seu arquivo shp - no SRC WGS 84
aoi = gpd.read_file(shapefile_path).to_crs(epsg=4326)
aoi_geojson = mapping(aoi.geometry.union_all())

In [8]:
# 3. Parâmetros
start_date = datetime(2018, 5, 1) # Data de início da sua série
end_date = datetime(2025, 5, 13) # Data de término da série
max_cloud = 35 # Defina a cobertura de nuvens máxima da imagem
scale = 10  # Resolução em m²/pixel

##Funções Principais

In [9]:
# 4. Função para calcular índices - Altere aqui com base nos índices que deseja calcular
def calcular_indices(datacube):
    ndvi = (datacube.band("B08") - datacube.band("B04")) / (datacube.band("B08") + datacube.band("B04"))
    ndre = (datacube.band("B08") - datacube.band("B05")) / (datacube.band("B08") + datacube.band("B05"))
    ndwi1 = (datacube.band("B11") - datacube.band("B08")) / (datacube.band("B11") + datacube.band("B08"))
    ndwi2 = (datacube.band("B12") - datacube.band("B08")) / (datacube.band("B12") + datacube.band("B08"))
    lnc = ((datacube.band("B08") - datacube.band("B05")) / (datacube.band("B08") + datacube.band("B05"))) * 4.060 + 0.43
    mcari = ((datacube.band("B05") - datacube.band("B04") * 0.2 - datacube.band("B03")) * datacube.band("B05")) / datacube.band("B04")
    mcari1 = ((datacube.band("B08") - datacube.band("B04")) * 2.5 - (datacube.band("B08") - datacube.band("B03")) * 1.3) * 1.2
    gndvi = (datacube.band("B08") - datacube.band("B03")) / (datacube.band("B08") + datacube.band("B03"))
    gosavi = 1.16 * ((datacube.band("B08") - datacube.band("B03")) / (datacube.band("B08") + datacube.band("B03") + 0.16))
    return {
        "NDVI": ndvi,
        "NDRE": ndre,
        "NDWI1": ndwi1,
        "NDWI2": ndwi2,
        "LNC": lnc,
        "MCARI": mcari,
        "MCARI1": mcari1,
        "GNDVI": gndvi,
        "GOSAVI": gosavi
    }

In [ ]:
# 5. Loop mensal - Define o intervalo para o download das imagens, no código atual é baixada 1 imagem por mês, selecionando a melhor imagem com base na cobertura de imagens total
current = start_date
while current < end_date:
    start_str = current.strftime("%Y-%m-%d")
    year_month = current.strftime("%Y-%m")
    print(f"\n🔄 Processando mês: {year_month}...")

    try:
        datacube = conn.load_collection(
            "SENTINEL2_L2A",
            spatial_extent=aoi_geojson,
            temporal_extent=[start_str, (current + timedelta(days=31)).replace(day=1).strftime("%Y-%m-%d")],
            bands=["B03", "B04", "B05", "B08", "B11", "B12"],
            max_cloud_cover=max_cloud
        )
    except:
        print("⚠️ L2A não disponível. Usando L1C.")
        datacube = conn.load_collection(
            "SENTINEL2_L1C",
            spatial_extent=aoi_geojson,
            temporal_extent=[start_str, (current + timedelta(days=31)).replace(day=1).strftime("%Y-%m-%d")],
            bands=["B03", "B04", "B05", "B08", "B11", "B12"],
            max_cloud_cover=max_cloud
        )

    datacube = datacube.reduce_dimension(dimension="t", reducer="mean")
    indices = calcular_indices(datacube)

    for nome, banda in indices.items():
        folder = f"/content/drive/MyDrive/Dissertacao/indices/ouro_fino/openEO/{nome}"
        os.makedirs(folder, exist_ok=True)

        # Nome final do arquivo com índice e data
        filename = f"{nome}_{year_month}.tif"
        filepath = os.path.join(folder, filename)

        # Verifica se o arquivo já existe
        if os.path.exists(filepath):
            print(f"⏭️ {filename} já existe. Pulando...")
            continue

        print(f"📥 Iniciando download de {filename}...")

        try:
            result = banda.save_result(format="GTiff")
            job = result.create_job(title=filename)
            job.start_and_wait()

            # Lista arquivos antes do download
            arquivos_antes = set(os.listdir(folder))

            # Faz o download
            job.download_results(target=folder)

            # Lista arquivos após o download
            arquivos_depois = set(os.listdir(folder))
            novos_arquivos = arquivos_depois - arquivos_antes

            novos_tifs = [f for f in novos_arquivos if f.endswith(".tif")]

            if novos_tifs:
                downloaded_path = os.path.join(folder, novos_tifs[0])
                shutil.move(downloaded_path, filepath)
                print(f"✅ {filename} salvo com sucesso.")
            else:
                print(f"⚠️ Nenhum arquivo .tif novo encontrado após download para {nome} em {year_month}.")

        except Exception as e:
            print(f"❌ Erro ao processar {filename}: {e}")

    current = (current + timedelta(days=31)).replace(day=1)

print("\n✅ Todos os meses foram processados com sucesso!")



🔄 Processando mês: 2018-05...
⏭️ NDVI_2018-05.tif já existe. Pulando...
⏭️ NDRE_2018-05.tif já existe. Pulando...
⏭️ NDWI1_2018-05.tif já existe. Pulando...
⏭️ NDWI2_2018-05.tif já existe. Pulando...
⏭️ LNC_2018-05.tif já existe. Pulando...
⏭️ MCARI_2018-05.tif já existe. Pulando...
⏭️ MCARI1_2018-05.tif já existe. Pulando...
⏭️ GNDVI_2018-05.tif já existe. Pulando...
⏭️ GOSAVI_2018-05.tif já existe. Pulando...

🔄 Processando mês: 2018-06...
⏭️ NDVI_2018-06.tif já existe. Pulando...
⏭️ NDRE_2018-06.tif já existe. Pulando...
⏭️ NDWI1_2018-06.tif já existe. Pulando...
⏭️ NDWI2_2018-06.tif já existe. Pulando...
⏭️ LNC_2018-06.tif já existe. Pulando...
⏭️ MCARI_2018-06.tif já existe. Pulando...
⏭️ MCARI1_2018-06.tif já existe. Pulando...
⏭️ GNDVI_2018-06.tif já existe. Pulando...
⏭️ GOSAVI_2018-06.tif já existe. Pulando...

🔄 Processando mês: 2018-07...
⏭️ NDVI_2018-07.tif já existe. Pulando...
⏭️ NDRE_2018-07.tif já existe. Pulando...
⏭️ NDWI1_2018-07.tif já existe. Pulando...
⏭️ NDWI2_2

<ipython-input-10-60f5f3c9b0f6>:53: UserDeprecationWarning: Call to deprecated method download_results. (Instead use `BatchJob.get_results` and the more flexible download functionality of `JobResults`) -- Deprecated since version 0.4.10.
  job.download_results(target=folder)
/usr/local/lib/python3.11/dist-packages/openeo/rest/job.py:174: UserDeprecationWarning: Call to deprecated method get_result. (Use `BatchJob.get_results` instead.) -- Deprecated since version 0.4.10.
  return self.get_result().download_files(target)
/usr/local/lib/python3.11/dist-packages/openeo/rest/job.py:178: UserDeprecationWarning: Call to deprecated class _Result. (Use `JobResults` instead) -- Deprecated since version 0.4.10.
  return _Result(self)


✅ NDVI_2018-11.tif salvo com sucesso.
📥 Iniciando download de NDRE_2018-11.tif...
0:00:00 Job 'j-25051613431840a980d11bdd4e8c3f3c': send 'start'
0:00:13 Job 'j-25051613431840a980d11bdd4e8c3f3c': created (progress 0%)
0:00:18 Job 'j-25051613431840a980d11bdd4e8c3f3c': created (progress 0%)
0:00:25 Job 'j-25051613431840a980d11bdd4e8c3f3c': created (progress 0%)
0:00:33 Job 'j-25051613431840a980d11bdd4e8c3f3c': created (progress 0%)
0:00:43 Job 'j-25051613431840a980d11bdd4e8c3f3c': created (progress 0%)
0:00:55 Job 'j-25051613431840a980d11bdd4e8c3f3c': running (progress N/A)
0:01:11 Job 'j-25051613431840a980d11bdd4e8c3f3c': running (progress N/A)
0:01:30 Job 'j-25051613431840a980d11bdd4e8c3f3c': running (progress N/A)
0:01:55 Job 'j-25051613431840a980d11bdd4e8c3f3c': finished (progress 100%)
✅ NDRE_2018-11.tif salvo com sucesso.
📥 Iniciando download de NDWI1_2018-11.tif...
0:00:00 Job 'j-2505161345164f69b4e3c749e3d58949': send 'start'
0:00:12 Job 'j-2505161345164f69b4e3c749e3d58949': creat